# Testing NOBIL data

### connecting with duckdb

In [130]:
import duckdb
from pathlib import Path


path_duckdb = Path.cwd().parent / "data_warehouse"

with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    cursor = conn.execute("SELECT * FROM staging.csmd_table_dump;")
    df_csmd = cursor.df()


df_csmd.head()


,id,name,ocpidb_mapping_stasjon_id,street,house_number,zipcode,city,municipality_id,municipality,county_id,...,created,updated,station_status,land_code,international_id,_dlt_load_id,_dlt_id,description_of_location,user_comment,contact_info
0,14671,Agnesfridsvägen 193,SE#VAT#75dbcba6-8bdf-4c70-9b5e-c7e2ab4dd50d,Agnesfridsvägen,193,21375,Malmö,1280,Malmö,12,...,2023-04-24 11:31:42+02:00,2025-09-21 01:13:17+02:00,1,SWE,SWE_14671,1765794546.3968723,W7Z70oeFL/dK3w,None,None,None
1,54376,Reload Filipstad,SE#VIR#SEVIRLJX9T0WRY,John Ericssonsgatan,8,68230,Filipstad,1782,Filipstad,17,...,2025-06-13 03:44:32+02:00,2025-06-14 03:44:28+02:00,1,SWE,SWE_54376,1765794546.3968723,5s2dmDY6IJpnqA,None,None,None
2,36573,Källhage Fast AB,SE#CNE#10e955c3-9683-4e7a-de1f-08dc39d02872,Toredalsgatan,3,41871,Göteborg,1480,Göteborg,14,...,2024-11-02 00:00:34+01:00,2025-11-16 20:18:09+01:00,1,SWE,SWE_36573,1765794546.3968723,AHTJ/g6BK4mQdw,None,None,None
3,54401,Jönköpings kommun - P-hus Smedjan,SE#VLT#6cbb6162-2630-40ef-a66d-44a91bf9a6ed,Södra Strandgatan,59,55320,Jönköping,0680,Jönköping,06,...,2025-06-13 07:00:56+02:00,2025-06-13 07:00:56+02:00,1,SWE,SWE_54401,1765794546.3968723,/h051P+jnFh0Eg,None,None,None
4,18856,500,DK#MON#s384017,Flygarvägen,37,17569,Järfälla,0123,Järfälla,01,...,2023-12-15 12:57:07+01:00,2025-11-18 05:22:17+01:00,1,SWE,SWE_18856,1765794546.3968723,KC194rk0Z49FNg,None,None,None


In [ ]:
for kommun in df_csmd["municipality"].unique():
    print(kommun)

In [131]:
with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    cursor = conn.execute("SELECT * FROM staging.status_online_table_dump;")
    df_st = cursor.df()
    
df_st.head()

,station_id,updated_date,attr_key,attrtypeid,attrname,attrvalid,trans,attrval,_dlt_load_id,_dlt_id
0,14671,2025-09-21 01:13:17+02:00,24,24,Open 24h,1,Yes,1,1765794546.3968723,3CKnsG93QiYBjQ
1,14671,2025-09-21 01:13:17+02:00,21,21,Real-time information,2,No,,1765794546.3968723,pn3ocTJ6mHpyGg
2,14671,2025-09-21 01:13:17+02:00,22,22,Public funding,4,None,,1765794546.3968723,bsTh6UvaSWdYCA
3,14671,2025-09-21 01:13:17+02:00,7,7,Parking fee,2,No,False,1765794546.3968723,WhlFMZ4/TLBYOA
4,14671,2025-09-21 01:13:17+02:00,6,6,Time limit,2,No,0,1765794546.3968723,uNOuJKH2Bww0hw


In [ ]:
with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    cursor = conn.execute("SELECT * FROM staging.connector_table_dump;")
    df_conn = cursor.df()
    
df_conn

In [ ]:
from datetime import date
import datetime

x = datetime.datetime.now()

today_date = date(x.year, x.month, x.day).isoformat()

print(today_date)


In [ ]:
import requests 
from constants.utils import API_KEY_NOBIL


params = {
    "apikey": API_KEY_NOBIL,
    "countrycode": "SWE",
    "format": "json",
    "file": "false",
}

url = "https://nobil.no/api/server/datadump.php"

r = requests.get(url=url, params=params)
r.raise_for_status()

In [ ]:
data = r.json()

In [ ]:
data["chargerstations"][0][""]

In [ ]:
from pathlib import Path
import pandas as pd
path = Path.cwd().parents[0]/"data"



df = pd.read_excel(path/"trafikanalys.xlsx", sheet_name= "Data")





In [ ]:
df.head()

In [ ]:

df = df.reset_index().rename({"index": "id"}, axis=1)
df["id"] = df["id"].apply(lambda x: x+1)




In [ ]:
df 

In [ ]:

df = df.to_dict(orient= "records")



In [ ]:
df

# Trafikanalys filter kommun och län


In [ ]:
import duckdb
from pathlib import Path


path_duckdb = Path.cwd().parent / "data_warehouse"

with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    
    df= conn.execute("select * from staging.trafikanalys;")
    
    df_trafikanalys = df.df()
    
    

In [ ]:

# ~df_trafikanalys, "~" betyder Not, så det som inte är lika med det i contains kommer det att printas ut
# astype för sätta data typ, str vektoresering för att använda strip på varje post, str med contains är vektoresera varje post och använder
# contains med regex, case är boolesk om det ska vara case sensitive och regex boolesk för tolka det man skickar in är regular expression. 
filter = df_trafikanalys[~df_trafikanalys["kommun"].astype(str).str.strip().str.contains(r"Okänd kommun|Totalt", case= False, regex= True, na= False)]



In [ ]:

kommuner = sorted(filter["kommun"].dropna().unique().tolist())






In [ ]:
import numpy as np
np.set_printoptions(threshold=np.inf)


print(kommuner)

In [ ]:

# Samma som för kommun fast nu är det för län
df_filter_län = df_trafikanalys[~df_trafikanalys["l_n"].astype(str).str.strip().str.contains(r"Okänt län|Totalt",case= False, regex= True, na=False)]





In [ ]:
df_filter_län

In [ ]:
län = sorted(df_filter_län["l_n"].dropna().unique().tolist())

In [ ]:
print(län)

# Create csv file in seed for kommun and län

In [ ]:



df_trafikanalys = df_trafikanalys[~df_trafikanalys["kommun"].astype(str).str.strip().str.contains(r"Okänd kommun|Totalt", case= False, regex= True, na= False)]
df_trafikanalys = df_trafikanalys[~df_trafikanalys["l_n"].astype(str).str.strip().str.contains(r"Okänt län|Totalt",case= False, regex= True, na=False)]



In [ ]:

# Create csv file in seeds
path = Path().cwd().parent/"data_transform/seeds"

df_kommun_lan = (df_trafikanalys[["kommun", "l_n"]].drop_duplicates())

df_kommun_lan.to_csv(path/"municipality_lan.csv", index=False, encoding= "utf-8")


In [ ]:
len(df_kommun_lan["l_n"].unique())